In [ ]:
import requests
import pandas as pd
import os
import sys
from decimal import Decimal, getcontext

In [ ]:
def filter_assetTransferred_logs(row):
    expanded_data = parse_decoded_input(row['decoded'])
    if not expanded_data.empty:
        return expanded_data
    return pd.Series(dtype='float')  # Consistent empty return

def parse_decoded_input(input_dict):
    if input_dict.get('method_call', '').startswith('AssetTransferred'):
        expanded_data = {'method_call': input_dict['method_call']}
        # print(input_dict.get('parameters', []))
        for param in input_dict.get('parameters', []):
            expanded_data[param['name']] = param['value']
        return pd.Series(expanded_data)
    return pd.Series(dtype='float')

def from_bn(val, dec=18):
    # Set the precision for Decimal operations (optional, adjust as needed)
    getcontext().prec = 50
    
    # Convert val to Decimal, assuming val can be cast directly
    if isinstance(val, str):
        val = Decimal(val)
    elif not isinstance(val, Decimal):
        val = Decimal(str(val))
    
    # Define the divisor as 10 raised to the power of dec
    divisor = Decimal(10) ** dec
    
    # Perform the division
    result = val / divisor
    
    # Convert the result to a string
    return str(result)

def sub_id_to_option_details(sub_id):
    if isinstance(sub_id, str):
        sub_id = int(sub_id)
    # Define mask constants
    UINT32_MAX = 0xffffffff
    UINT63_MAX = 0x7fffffffffffffff

    # Bitwise operations to extract values
    expiry = sub_id & UINT32_MAX
    strike = ((sub_id >> 32) & UINT63_MAX) * 10_000_000_000
    is_call = (sub_id >> 95) > 0

    # Return the details as a dictionary (could also use a custom class or namedtuple)
    # expiry is a timestamp in seconds, convert it to a date object which is more readable
    return dict(
        expiry = pd.to_datetime(expiry, unit='s'),
        strike = from_bn(strike),
        type= 'call' if is_call else 'put'
    )

def extract_option_details(logs):
    # Ensure the columns 'expiry', 'strike', and 'type' exist in the DataFrame
    if 'expiry' not in logs.columns:
        logs['expiry'] = None
    if 'strike' not in logs.columns:
        logs['strike'] = None
    if 'type' not in logs.columns:
        logs['type'] = None
        
    for index, row in logs.iterrows():
        sub_id = row['subId']
        if sub_id != '0':
            option_details = sub_id_to_option_details(sub_id)
            # attach option details['expiry'] to the row
            logs.at[index, 'expiry'] = option_details['expiry']
            logs.at[index, 'strike'] = option_details['strike']
            logs.at[index, 'type'] = option_details['type']
    return logs

In [ ]:
columns_to_keep = ['fromAcc', 'toAcc', 'asset', 'subId', 'amount', 'tradeId', 'expiry', 'strike', 'type', 'block_number', 'index', 'tx_hash','method_call',]
def fetch_all_logs(n, next_page_params, logs=pd.DataFrame()):    
    url = 'https://explorer.lyra.finance/api/v2/addresses/0xE7603DF191D699d8BD9891b821347dbAb889E5a5/logs'
    headers = {'accept': 'application/json'}
    from_block = next_page_params['block_number']
    for i in range(n):
        response = requests.get(url, headers=headers, params=next_page_params)
        if response.status_code == 200:
            data = response.json()
            batch = pd.DataFrame(data['items'])
            # print(batch)
            new_rows = batch.apply(filter_assetTransferred_logs, axis=1)
            # Directly filter rows where new_rows are not empty before concatenating

            # Ensure 'method_call' column exists in new_rows
            if 'method_call' not in new_rows.columns:
                new_rows['method_call'] = None

            mask = new_rows['method_call'].notnull()
            df_combined = batch[mask].join(new_rows[mask])  # Apply mask directly here

            # Ensure the necessary columns exist in df_combined
            for col in columns_to_keep:
                if col not in df_combined.columns:
                    df_combined[col] = None

            parsed = extract_option_details(df_combined)[columns_to_keep]
            logs = pd.concat([logs, parsed])
            print("{}: Fetched {} asset transfer logs, next page {}".format(i, len(parsed), next_page_params))
            next_page_params = data['next_page_params']
            c = next_page_params['block_number']
            if(i%100 == 0):
                print(f'saving {i}')   
                logs.to_csv(f'./extracted/log-{from_block}-{c}.csv', index=False)
        else:
            print("Failed to fetch data:", response.json())
            fetch_all_logs(n, next_page_params)
    return logs

In [ ]:
import time
max_retries = 100  # Maximum number of retries
retry_delay = 3  # Delay between retries in seconds

retry_count = 0
successful = False

while not successful and retry_count < max_retries:
    try:
        # Read all CSV files in the 'extracted' directory and concatenate them
        sofar = pd.concat([pd.read_csv(f'extracted/{f}') for f in os.listdir('extracted') if f.endswith('.csv')])
        sofar.drop_duplicates(inplace=True)
        sofar.reset_index(drop=True, inplace=True)

        # Find the row with the minimum 'block_number'
        last_block_index = sofar['block_number'].idxmin()
        next_page_params = {
            'block_number': sofar.iloc[last_block_index]['block_number'],
            'index': sofar.iloc[last_block_index]['index'],
            'items_count': 450
        }

        print('Starting from block:', next_page_params)

        # 10000 is roughly a month
        logs = fetch_all_logs(20000, next_page_params)
        successful = True  # Set to True if everything was successful

    except Exception as e:
        print(f'Error occurred: {e}')
        retry_count += 1
        print(f'Retrying... Attempt {retry_count}/{max_retries}')
        time.sleep(retry_delay)  # Wait before retrying

if not successful:
    print("Failed to complete the process after several retries.")

In [ ]:
# Read all CSV files in the 'extracted' directory and concatenate them
all_extracted_logs = pd.concat([pd.read_csv(f'extracted/{f}') for f in os.listdir('extracted') if f.endswith('.csv')])
all_extracted_logs.drop_duplicates(inplace=True)
all_extracted_logs.reset_index(drop=True, inplace=True)
last_block_index = all_extracted_logs.iloc[all_extracted_logs['block_number'].idxmin()]['block_number']
first_block_index = all_extracted_logs.iloc[all_extracted_logs['block_number'].idxmax()]['block_number']
# add them to blockkchain logs dir to process
all_extracted_logs.to_csv(f'blockchain logs/transfers.csv')